In [2]:
import panel as pn
import os
import numpy as np
import bokeh

from bokeh.layouts import column, row, Spacer
from bokeh.models import CustomJS, Slider, RadioButtonGroup, TextInput, Button, MultiChoice
from bokeh.models import BoxAnnotation, PreText, Range1d, LinearAxis, Span, HoverTool, DataTable, TableColumn
from bokeh.events import SelectionGeometry
from bokeh.plotting import ColumnDataSource, figure, show

!python --version
!which python

Python 3.8.8
/home/jhoskins/anaconda3/bin/python


In [ ]:
import sys
sys.path.append('/home/jhoskins/anaconda3/lib/python3.8/site-packages')

In [ ]:
print(sys.path)

In [ ]:
!/home/jhoskins/anaconda3/bin/python -m pip install casatasks==6.2.0.124
!/home/jhoskins/anaconda3/bin/python -m pip install casatools==6.2.0.124
!/home/jhoskins/anaconda3/bin/python -m pip install casatestutils==6.2.0.124

In [ ]:
import casatasks
import casatools

In [9]:
import yaml

class TCleanOptions:
    def __init__(self, config_file:str):
        self.config_file: str = config_file
        
        self.vis: str 
        self.imagename: str 
        self.imsize: int 
        self.cell: str 
        self.specmode: str
        self.interpolation: str
        self.nchan: int
        self.start: str
        self.width: str
        self.pblimit: float
        self.deconvolver: str
        self.niter: int
        self.cyclefactor: int
        self.scales: list
        self.interactive: int
    
    def read_configuration(self)->None:
        with open(self.config_file) as file:
            config = yaml.full_load(file)
        
        self.vis = config['tclean']['vis']
        self.imagename = config['tclean']['imagename']
        self.imsize = config['tclean']['imsize']
        self.cell = config['tclean']['cell']
        self.specmode = config['tclean']['specmode']
        self.interpolation = config['tclean']['interpolation']
        self.nchan = config['tclean']['nchan']
        self.start = config['tclean']['start']
        self.width = config['tclean']['width']
        self.pblimit = config['tclean']['pblimit']
        self.deconvolver = config['tclean']['deconvolver']
        self.niter = config['tclean']['niter']
        self.cyclefactor = config['tclean']['cyclefactor']
        self.scales = config['tclean']['scales']
        self.interactive = config['tclean']['interactive']


In [10]:
options = TCleanOptions('stakeholder_tools/config/tclean.yaml')
options.read_configuration()

In [ ]:
pn.extension()

In [ ]:
class TCleanPanel():
    
    def __init__(self):
        OPTIONS = [('I', 'I'), ('Q', 'Q'), ('U', 'U'), ('V', 'V')]
        
        self.vis = pn.widgets.TextInput(
            value="select ms file ...", 
            name="Vis")

        self.imagename = TextInput(
            value="output file name ...", 
            title="Imagename")
        
        self.imsize = pn.widgets.IntInput(
            name='imsize', 
            value=512, 
            step=1, 
            start=1, 
            end=1000)

        self.cell = pn.widgets.TextInput(
            value="8.0arcsec", 
            name="Cell")

        self.specmode = pn.widgets.TextInput(
            value="cube", 
            name="Specmode")
        
        self.interpolation = pn.widgets.TextInput(
            value="nearest", 
            name="Interpolation")
        
        self.nchan = pn.widgets.IntInput(
            name='nchan', 
            value=5, 
            step=1, 
            start=1, 
            end=500)

        self.start = pn.widgets.TextInput(
            value="1.0GHz", 
            name="Start")

        self.width = pn.widgets.TextInput(
            value="0.2GHz", 
            name="Width")
        
        self.pblimit = pn.widgets.FloatInput(
            name='pblimit', 
            value=-0.00001, 
            step=0.000001, 
            start=-0.000001, 
            end=-0.0001)

        self.deconvolver = pn.widgets.TextInput(
            value="hogbom", 
            name="Deconvolver")
        
        self.niter = pn.widgets.IntInput(
            name='niter', 
            value=5, 
            step=1, 
            start=1, 
            end=5000)
        
        self.cyclefactor = pn.widgets.IntInput(
            name='cyclefactor', 
            value=1, 
            step=1, 
            start=1, 
            end=500)
        
        self.interactive = pn.widgets.IntInput(
            name='interactive', 
            value=0, 
            step=1, 
            start=0, 
            end=1)

    
    # ------
    
        self.play_button = pn.widgets.Button(
            name="Play", 
            button_type="success",  
            margin=(5,1,5,1)) 
        #icon=SVGIcon(icon_name="play"))
    
        self.layout = pn.Row(
            pn.Column(
                self.vis,
                self.imagename,
                self.imsize,
                self.cell,
                self.specmode,
                self.interpolation,
                self.nchan,
                self.start,
                self.width,
                self.pblimit,
                self.deconvolver,
                self.niter,
                self.cyclefactor,
                self.interactive,
                self.play_button    
            )
        )

In [ ]:
tclean = TCleanPanel()

In [ ]:
tclean.layout

In [ ]:
casatasks.tclean(
    vis='../data/sim_data_VLA.ms', 
    imagename='test', 
    imsize=options.imsize, 
    cell=options.cell, 
    specmode=options.specmode, 
    interpolation=options.interpolation, 
    nchan=options.nchan, 
    start=options.start, 
    width=options.width, 
    pblimit=options.pblimit, 
    deconvolver=options.deconvolver, 
    niter=options.niter, 
    cyclefactor=options.cyclefactor, 
    scales=options.scales, 
    interactive=0)